In [5]:
import streamlit as st
import pandas as pd
import re
import string
import json
from nltk.tokenize import word_tokenize as token_kata
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


In [7]:
file_path = "csv/data_tweet.csv"  # Lokasi file CSV data awal
with open(file_path, "r", encoding="utf-8") as f:
    csv_raw_data = f.read()

In [8]:
# KUMPULAN FUNGSI PREPROSESING

# Fungsi untuk mengubah teks menjadi huruf kecil (lowercase)
def ubah_ke_huruf_kecil(dataframe, column_name):
    dataframe[column_name] = dataframe[column_name].str.lower()
    return dataframe

# Fungsi untuk menghapus karakte-karakter spesial twitter(X) dari data hasil scrapping
def bersihkan_karakter_twitter(text):
    text = text.replace('\\t', " ").replace('\\n', " ").replace('\\u', " ").replace('\\', "")
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = ' '.join(re.sub(r"([@#][A-Za-z0-9_]+)|(\w+:\/\/\S+)", " ", text).split())
    return text.replace("http://", " ").replace("https://", " ")

# Fungsi untuk menghapus angka untuk penyederhanaan data clear
def hapus_angka(text):
    return re.sub(r"\d+", "", text)

# Fungsi untuk menghapus tanda baca seperti titik, koma, tanda seru, dll.
def hapus_tandabaca(text):
    return text.translate(str.maketrans("", "", string.punctuation))

# Fungsi untuk menghapus whitespace atau spasi yang ada di awal dan akhir teks.
def hapus_spasi_awalakhir(text):
    return text.strip()

# Fungsi untuk mengganti spasi berturut turut dengan satu spasi tunggal.
def ganti_spasi_tunggal(text):
    return re.sub(r'\s+', ' ', text)

# Fungsi untuk menghapus karakter tunggal yang berdiri sendiri di dalam teks
def hapus_karakter_tunggal(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

# Fungsi untuk membaca file JSON yang berisi kamus slang (bahasa gaul) dan menyimpannya ke dalam bentuk dictionary
def muat_kamus_slang(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)
    
# Fungsi untuk menggantikan bahasa gaul di dalam teks dengan padanan yang lebih baku berdasarkan kamus slang.
def slang_ke_baku(text, kamus_bahasa_gaul):
    return " ".join(kamus_bahasa_gaul.get(word, word) for word in text.split())

# Fungsi untuk memecah teks menjadi token atau kata kata individual
def bungkus_tokenisasi_kata(text):
    return token_kata(text)


class StopWordsIndo:
    def __init__(self, stopwords_file):
        self.stopwords = self.olah_stopword(stopwords_file)
    
    def olah_stopword(self, stopwords_file):
        with open(stopwords_file, 'r', encoding='utf-8') as f:
            return set(f.read().splitlines())
    
    def hapus_stopwords(self, text):
        pisahkan_kata = text.split()
        kata_bersih = [word for word in pisahkan_kata if word not in self.stopwords and len(word) > 3]
        return " ".join(kata_bersih)
class KamusFilter:
    def __init__(self, kamus_file):
        self.term_dict = self.baca_kamus(kamus_file)

    def baca_kamus(self, kamus_file):
        try:
            with open(kamus_file, 'r', encoding='utf-8') as file:
                return set(file.read().splitlines())
        except FileNotFoundError:
            print(f"File {kamus_file} tidak ditemukan.")
            return set()

    def hapus_bukan_id(self, document):
        return [term for term in document if term in self.term_dict]

# Stemming
pengolahdata = StemmerFactory() # Digunakan untuk membuat objek stemmer.
stemmer = pengolahdata.create_stemmer() # Membuat objek stemmer yang dapat digunakan untuk melakukan proses stemming.

# Fungsi untuk mengubah setiap kata ke bentuk dasarnya.
def sederhanakan_teks(text):
    return stemmer.stem(text)

# Fungsi untuk memuat leksikon sentimen
def load_lexicon(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Fungsi untuk melakukan analisis sentimen pada sebuah teks
def sentiment_analysis(text, pos_lexicon, neg_lexicon):
    words = text.split()
    pos_count = sum(1 for word in words if word in pos_lexicon)
    neg_count = sum(1 for word in words if word in neg_lexicon)
    
    if pos_count > neg_count:
        return 'Positif', pos_count, neg_count
    elif neg_count > pos_count:
        return 'Negatif', pos_count, neg_count
    else:
        return 'Netral', pos_count, neg_count

In [9]:
def preprocess_data(uploaded_file):
    corpus_df = pd.read_csv(uploaded_file)
    corpus_df = ubah_ke_huruf_kecil(corpus_df, 'full_text')
    corpus_df['full_text'] = corpus_df['full_text'].apply(bersihkan_karakter_twitter)
    corpus_df['full_text'] = corpus_df['full_text'].apply(hapus_angka)
    corpus_df['full_text'] = corpus_df['full_text'].apply(hapus_tandabaca)
    corpus_df['full_text'] = corpus_df['full_text'].apply(hapus_spasi_awalakhir)
    corpus_df['full_text'] = corpus_df['full_text'].apply(ganti_spasi_tunggal)
    corpus_df['full_text'] = corpus_df['full_text'].apply(hapus_karakter_tunggal)

    # Load dan replace slang
    kamus_bahasa_gaul = muat_kamus_slang("txt/kamusSlang.json")
    corpus_df['full_text'] = corpus_df['full_text'].apply(lambda x: slang_ke_baku(x, kamus_bahasa_gaul))

    # Tokenisasi
    corpus_df['tokenisasi'] = corpus_df['full_text'].apply(bungkus_tokenisasi_kata)

    # Stemming
    corpus_df['stemmed'] = corpus_df['full_text'].apply(sederhanakan_teks)

    # Inisialisasi stopword dan kamus filter
    stopwords_processor = StopWordsIndo('txt/stopwords.txt')
    kamus_filter = KamusFilter("txt/kamusIndonesia.txt")

    # Hapus stopword
    corpus_df['stopwords'] = corpus_df['stemmed'].apply(lambda x: stopwords_processor.hapus_stopwords(x))

    # Filter term non-indonesia
    corpus_df['filtered'] = corpus_df['stopwords'].apply(lambda x: kamus_filter.hapus_bukan_id(x.split()))

    # Menyimpan hasil preproses kedalam file 'hasil.txt'
    corpus_df.to_csv('preprosesing/hasil.txt', index=None, header=True)
    return corpus_df

In [10]:
import pandas as pd
import json

file_path = "csv/data_tweet.csv"  # Lokasi file CSV data awal
corpus_df = pd.read_csv(file_path, encoding="utf-8")  
print(corpus_df)


                                              full_text
0     toxic trait ku adalah : suka takut ditinggalin...
1     Inilah 5 Perilaku Toxic yang sering ditemui da...
2     Seperti yang dibahas oleh oknum r. Saya juga m...
3     Bahayanya hubungan toxic itu kamu ga sadar uda...
4     salah satu bahayanya hubungan Toxic adalah kam...
...                                                 ...
2022  @Dennyhrt Sakit si engga. Cuman agak pegel aja...
2023  lagian ni orang lagi ada masalah malah berulah...
2024  Cubalah nyatakan apa yg menyebabkan proses men...
2025  @bfgrent @devoztion ISENG TENGIL dan tipikal p...
2026  Anjeng ini psikologi yang dipelajari apa aja w...

[2027 rows x 1 columns]


In [17]:
def ubah_ke_huruf_kecil(dataframe, column_name):
    dataframe[column_name] = dataframe[column_name].str.lower()
    return dataframe

# Fungsi untuk membaca file JSON yang berisi kamus slang (bahasa gaul) dan menyimpannya ke dalam bentuk dictionary
def muat_kamus_slang(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)
    
# Fungsi untuk menggantikan bahasa gaul di dalam teks dengan padanan yang lebih baku berdasarkan kamus slang.
def slang_ke_baku(text, kamus_bahasa_gaul):
    return " ".join(kamus_bahasa_gaul.get(word, word) for word in text.split())

corpus_df = ubah_ke_huruf_kecil(corpus_df, 'full_text')
corpus_df['full_text'] = corpus_df['full_text'].apply(bersihkan_karakter_twitter)
corpus_df['full_text'] = corpus_df['full_text'].apply(hapus_angka)
corpus_df['full_text'] = corpus_df['full_text'].apply(hapus_tandabaca)
corpus_df['full_text'] = corpus_df['full_text'].apply(hapus_spasi_awalakhir)
corpus_df['full_text'] = corpus_df['full_text'].apply(ganti_spasi_tunggal)
corpus_df['full_text'] = corpus_df['full_text'].apply(hapus_karakter_tunggal)


kamus_bahasa_gaul = muat_kamus_slang("txt/kamusSlang.json")
corpus_df['full_text'] = corpus_df['full_text'].apply(lambda x: slang_ke_baku(x, kamus_bahasa_gaul))

# Tokenisasi
corpus_df['tokenisasi'] = corpus_df['full_text'].apply(bungkus_tokenisasi_kata)

# Stemming
corpus_df['stemmed'] = corpus_df['full_text'].apply(sederhanakan_teks)



In [20]:
# Inisialisasi stopword dan kamus filter
stopwords_processor = StopWordsIndo('txt/stopwords.txt')
kamus_filter = KamusFilter("txt/kamusIndonesia.txt")

# Hapus stopword
corpus_df['stopwords'] = corpus_df['stemmed'].apply(lambda x: stopwords_processor.hapus_stopwords(x))

# Filter term non-indonesia
corpus_df['filtered'] = corpus_df['stopwords'].apply(lambda x: kamus_filter.hapus_bukan_id(x.split()))

In [21]:
corpus_df.head(15)

,full_text,tokenisasi,stemmed,stopwords,filtered
0,jahat trait ku adalah suka takut ditinggalin a...,"[jahat, trait, ku, adalah, suka, takut, diting...",jahat trait ku adalah suka takut ditinggalin a...,jahat trait suka takut ditinggalin putus dulua...,"[jahat, suka, takut, putus, pergi]"
1,inilah perilaku jahat yang sering ditemui dan ...,"[inilah, perilaku, jahat, yang, sering, ditemu...",ini perilaku jahat yang sering temu dan normal...,perilaku jahat temu normalisasi simak ulas thread,"[perilaku, jahat, temu, normalisasi, simak, ulas]"
2,seperti yang dibahas oleh oknum saya juga mera...,"[seperti, yang, dibahas, oleh, oknum, saya, ju...",seperti yang bahas oleh oknum saya juga rasa h...,bahas oknum rasa hubung jahat lelah rasa waktu...,"[bahas, oknum, rasa, hubung, jahat, lelah, ras..."
3,bahayanya hubungan jahat itu kamu tidak sadar ...,"[bahayanya, hubungan, jahat, itu, kamu, tidak,...",bahaya hubung jahat itu kamu tidak sadar sudah...,bahaya hubung jahat sadar direndahin terusmene...,"[bahaya, hubung, jahat, sadar, rasa, rendah, r..."
4,salah satu bahayanya hubungan jahat adalah kam...,"[salah, satu, bahayanya, hubungan, jahat, adal...",salah satu bahaya hubung jahat adalah kamu tid...,salah bahaya hubung jahat sadar rendah rasa re...,"[salah, bahaya, hubung, jahat, sadar, rendah, ..."
5,sunners yuk mari kita kenali beberapa ciriciri...,"[sunners, yuk, mari, kita, kenali, beberapa, c...",sunners yuk mari kita kenal beberapa ciriciri ...,sunners mari kenal ciriciri hubung jahat pasan...,"[mari, kenal, hubung, jahat, pasang, atur, hid..."
6,bahayanya hubungan jahat itu kamu tidak sadar ...,"[bahayanya, hubungan, jahat, itu, kamu, tidak,...",bahaya hubung jahat itu kamu tidak sadar sudah...,bahaya hubung jahat sadar direndahin rasa rend...,"[bahaya, hubung, jahat, sadar, rasa, rendah, r..."
7,ciriciri orang jahat dan radflag egois suka ce...,"[ciriciri, orang, jahat, dan, radflag, egois, ...",ciriciri orang jahat dan radflag egois suka ce...,ciriciri jahat radflag egois suka cemburu sika...,"[jahat, egois, suka, cemburu, sikap, kasar, ni..."
8,bahayanya hubungan jahat itu kamu tidak sadar ...,"[bahayanya, hubungan, jahat, itu, kamu, tidak,...",bahaya hubung jahat itu kamu tidak sadar sudah...,bahaya hubung jahat sadar direndahin terusmene...,"[bahaya, hubung, jahat, sadar, rasa, rendah, r..."
9,ada istilah yang disebut jahat positivity yait...,"[ada, istilah, yang, disebut, jahat, positivit...",ada istilah yang sebut jahat positivity yaitu ...,istilah jahat positivity paksa bahagia optimis...,"[istilah, jahat, paksa, bahagia, optimis, situ..."


In [22]:
selected_columns = corpus_df[["stopwords", "filtered"]]
print(selected_columns.head(15))


                                            stopwords  \
0   jahat trait suka takut ditinggalin putus dulua...   
1   perilaku jahat temu normalisasi simak ulas thread   
2   bahas oknum rasa hubung jahat lelah rasa waktu...   
3   bahaya hubung jahat sadar direndahin terusmene...   
4   salah bahaya hubung jahat sadar rendah rasa re...   
5   sunners mari kenal ciriciri hubung jahat pasan...   
6   bahaya hubung jahat sadar direndahin rasa rend...   
7   ciriciri jahat radflag egois suka cemburu sika...   
8   bahaya hubung jahat sadar direndahin terusmene...   
9   istilah jahat positivity paksa bahagia optimis...   
10  risau jahat hidup pegang character jahat hidup...   
11                                              jahat   
12  ciri jahat relationship hubung kontrol pasang ...   
13  marah gara cium payah cium cium diam melotot s...   
14  tementemen tongkrong jahat entar jahat contoh ...   

                                             filtered  
0                  [jahat, suka